# 内容
```meeting_0122_0_0```の出力をプラスした版 ※回帰のまま


# したいこと
温度関係なく、エネルギーが0となったすべての```y```を格納したdataframeを作る

# 出力
* 各温度におけるエネルギーの推移
* 各温度での、全部のステップを経た後のy
* 各温度における最後の実験で、エネルギーが0となったものだけ表示されるdataframe
* 温度関係なく、エネルギーが0となったすべての```y```を格納したdataframe

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
#valiable
num_select = 3

MCMC_step = 1000
MCMC_step_each = 20

Max_temper = 5000
T = [j for j in range(1, Max_temper+1)]
len_T = len(T)

b = [1/ l for l in T]

In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
#print(data_yields)
ori_y = data_yields.T.iloc[0]
print(ori_y)

y= data_yields.T.iloc[0]

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [4]:
type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

#type(y[0]) #numpy.int64

(20,)

In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=f_regression, k=num_select) #SelectKBest #f_regression...線形回帰
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [7]:
def each_energy(mask_each): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [8]:
def y_renewal(s: int):
    return 1-s

In [9]:
E0_Y_lists_df = pd.DataFrame()
E0_Y_lists_df['ori_y']=y #初期設定のyを追加

In [10]:
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):
    E = []
    E.append(E_first)
    
    for l in range(MCMC_step_each):
        #print(j)
        j = random.randrange(num_feature)
        ori_yi = y[j]
        #print(y[j])
        y[j] = y_renewal(y[j])
        #print(y)
        mask_new = selection(num_select, X, y)
        #print(mask_new)
        E_now = selected_X_energy - each_energy(mask_new)
        #print(E_now)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[j] = ori_yi #元に戻す
        else:
            E.append(E_now)
            if E_now ==0:
                E0_Y_lists_df[len(E0_Y_lists_df.columns)] = y
    return E, y #エネルギーのリスト #最後のyだけ(毎回更新しているので)

In [11]:
E_lists = []
Y_lists = []
mask_ori = selection(num_select, X, ori_y)
print(mask_ori)
selected_X_energy = each_energy(mask_ori)
ori_y = data_yields.T.iloc[0]

[False False False False False  True False False False False False False
 False False False False False False  True  True]


In [12]:
for k in range(len_T):
    E, y = Montecarlo(T[k], num_select, MCMC_step_each, 0, ori_y) #0=each_energy(mask_ori, y)-each_energy(mask_ori, y)
    #print(y)
    E_lists.append(E) #はじめのE=0は入れていないぞ
    Y_lists.append(y)
    data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
    ori_y = data_yields.T.iloc[0]

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


In [13]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'
for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,0,1,1,1,0,0,1,0,0,1,...,1,0,1,0,0,1,1,1,1,1
1,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,0,1,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,0,0
3,1,1,0,1,1,1,0,1,1,1,...,0,1,0,0,0,0,0,0,0,1
4,0,0,1,0,1,0,1,0,1,0,...,1,0,1,1,1,1,1,1,1,1
5,0,0,0,0,0,0,0,1,1,1,...,0,0,0,1,1,0,1,0,0,0
6,1,1,1,1,0,0,0,1,1,0,...,1,0,0,0,1,1,1,0,1,0
7,1,1,0,0,1,0,0,0,0,1,...,0,1,1,1,1,0,0,1,1,1
8,0,1,0,0,0,0,0,0,1,0,...,1,1,0,1,1,1,1,0,0,0
9,1,1,1,1,1,0,0,0,1,0,...,1,0,1,1,1,0,1,1,0,1


In [14]:
E_lists_df = pd.DataFrame(index=[], columns=[])

#del E_lists[0]#元の条件削除⇒len(E_lists)=MCMC_step_each となるはず

i = random.randrange(len_T-1)
#r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
rnd = np.random.uniform(0.0, 1.0)
if rnd < r:
    E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]

In [15]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'
for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,0,1,1,1,0,0,1,0,0,1,...,1,0,1,0,0,1,1,1,1,1
1,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,0,1,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,0,0
3,1,1,0,1,1,1,0,1,1,1,...,0,1,0,0,0,0,0,0,0,1
4,0,0,1,0,1,0,1,0,1,0,...,1,0,1,1,1,1,1,1,1,1
5,0,0,0,0,0,0,0,1,1,1,...,0,0,0,1,1,0,1,0,0,0
6,1,1,1,1,0,0,0,1,1,0,...,1,0,0,0,1,1,1,0,1,0
7,1,1,0,0,1,0,0,0,0,1,...,0,1,1,1,1,0,0,1,1,1
8,0,1,0,0,0,0,0,0,1,0,...,1,1,0,1,1,1,1,0,0,0
9,1,1,1,1,1,0,0,0,1,0,...,1,0,1,1,1,0,1,1,0,1


In [16]:
# 各温度におけるエネルギーの推移
for j in range(int(MCMC_step/MCMC_step_each)):

    for k in range(len_T):
        E, y = Montecarlo(T[k], num_select, MCMC_step_each, E_lists[k][-1], Y_lists[k])
        E_lists.append(E)
        Y_lists.append(y)

    E_lists_df[j] = E_lists[0:len_T]
    del E_lists[0:len_T]#1つ前のループで作られたリストの削除
    del Y_lists[0:len_T]

    i = random.randrange(len_T-1)
    #r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
    r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
    rnd = np.random.uniform(0.0, 1.0)
    if rnd < r:
        E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]


#各温度でMCMC_step回動作させたことで、{MCMC_step_each個のエネルギーと最終y}のデータがkセットできた
#最後に知りたいのは、最終ykセットと、エネルギーの動き
E_lists_df.index.name = 'temperature'
E_lists_df.columns.name = 'each MCMC step'
E_lists_df

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

each MCMC step,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
temperature,,,,,,,,,,,,,,,,,,,,,
0,"[0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[4, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 4, 4]","[4, 4, 4, 4, 4, 4, 6, 2, 2, 6, 4, 2, 4, 4]","[4, 4, 4, 4, 4, 4, 6, 6, 4, 2, 2, 2, 2, 2, 4, ...","[2, 2, 2, 2, 0]","[0, 0, 0, 4, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",...,"[2, 4, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2]","[2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2]","[2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, ...","[4, 4, 4, 4, 4, 4, 2, 2, 2, 0, 2, 2, 2, 2, 2, ...","[4, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 2, 2]","[2, 4, 4, 4, 4, 4, 6, 6, 6, 2, 2]","[2, 2, 2, 2, 2, 2, 2]","[2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 4, 4, 4, 4, 4, 4, 4, 2]","[2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 2]"
1,"[0, 2, 2, 4, 6, 6]","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2, 4, 2]","[2, 4, 4, 4, 4, 4, 6, 6, 6, 4, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, ...","[4, 4, 6, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 6, ...","[6, 6, 6, 4, 4, 4, 4, 4, 6, 4, 4, 6, 4, 4, 4, ...","[4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, ...",...,"[6, 6, 4, 6, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 4, ...","[6, 6, 6, 6, 6, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, ...","[2, 4, 4, 4, 2, 4, 6, 4, 6, 6, 6, 6, 4, 4, 4, 4]","[4, 2, 4, 4, 4, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, ...","[4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4]","[4, 4, 4, 6, 6, 6, 6, 2]"
2,"[0, 2, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 6, ...","[4, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, ...","[4, 6, 4, 6, 6, 6, 4, 6, 4, 6, 2, 4, 2, 4, 4, ...","[4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 6, 4, 6, 6, 6, ...","[4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 6, 6, 4, ...","[4, 4, 4, 4, 6, 4, 6, 6, 4, 2, 2, 0, 2, 2, 2, ...","[2, 2, 0, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 6, 6, ...","[6, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 6, 4, 4, 4, ...","[4, 4, 4, 4, 2, 4, 2, 4, 4, 6, 6, 6, 6, 4, 4, ...",...,"[4, 4, 4, 2, 0, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 2, 4, ...","[2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 6, 4, 2, 4]","[4, 4, 2, 4, 6, 4, 6, 6, 6, 4, 6, 6, 6, 6, 6, ...","[6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 4, ...","[2, 2, 2, 2, 2, 2, 4, 4, 2, 4, 4, 2, 4, 2, 4, ...","[4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, ...","[4, 4, 4, 2, 6, 6, 4, 4, 4, 4, 2, 2, 2, 2, 2, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 6, 6]","[6, 6, 4, 4, 4, 6, 6, 6, 6, 4, 6, 4, 4, 4, 4, 4]"
3,"[0, 2, 2, 2, 2, 4, 4, 6, 4, 6, 6, 6, 6, 6, 6, ...","[4, 4, 6, 4, 4, 4, 4, 4, 6, 2, 2, 2, 4, 6, 6, ...","[4, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 6, 6, 6, ...","[2, 0, 4, 4, 6, 6, 6, 4, 4, 4, 4, 6, 6, 6, 6, ...","[6, 6, 6, 6, 6, 6, 4, 4, 4, 6, 4, 2, 4, 2, 4, 4]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[6, 6, 4, 4, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 4]","[4, 4, 4, 2, 2, 4, 4, 4, 2, 6, 4, 4, 4, 6, 4, ...","[2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, ...","[6, 6, 4, 4, 4, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, ...",...,"[4, 4, 4, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, ...","[6, 6, 6, 6, 4, 4, 6, 6, 6, 6, 6, 6, 4, 6, 4, ...","[6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 4, 4, 4, 4, ...","[6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[4, 4, 4, 4, 4, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, ...","[2, 2, 0, 0, 0, 2, 2, 6, 6, 6, 4, 6, 6, 6, 6, ...","[6, 6, 6, 6, 4, 6, 4, 4, 4, 4, 4, 4, 4, 6, 6, ...","[6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
4,"[0, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4,

In [17]:
# 各温度での、全部のステップを経た後のy
Y_lists_df = pd.DataFrame()
Y_lists_df.index.name = 'y'
Y_lists_df.columns.name = 'temperature'

for t in range(len_T):
    Y_lists_df[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df['ori']=ori_y
Y_lists_df

temperature,1,2,3,4,5,6,7,8,9,10,...,4992,4993,4994,4995,4996,4997,4998,4999,5000,ori
0,1,0,0,0,1,1,0,1,1,0,...,1,1,1,0,0,1,0,0,1,1
1,1,1,1,1,1,0,1,0,0,0,...,1,1,0,0,1,1,0,0,0,0
2,0,1,1,0,1,0,0,0,0,1,...,0,1,1,0,0,0,0,1,1,0
3,1,0,0,0,1,1,1,0,1,0,...,1,1,1,0,1,1,0,0,1,1
4,1,0,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,1,1,0,1
5,1,0,1,0,0,0,0,0,1,1,...,1,1,0,0,1,0,0,0,1,0
6,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
7,0,1,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
8,1,1,0,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,1,1,1,0,1


In [18]:
#E_lists[len_T - 1][-1] #IndexError: list index out of range
E_lists[0][-1] == 0 #False
E_lists[495][-1] == 0 #False
E_lists[16][-1] == 0 #True(t = 17)

none_count = 0
for t in range(len_T):
    if E_lists[t][-1] == None:
        none_count += 1

In [19]:
#エネルギーが0となったものyを示したdataframe
E0_Y_lists_df.index.name = 'y selected featrues are same as ori_y'
E0_Y_lists_df.T.drop_duplicates().T

,ori_y,1,3,4,5,8,10,11,12,13,...,17525,17661,17821,17839,17843,18103,18124,18214,18466,18686
y selected featrues are same as ori_y,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,1,0,1,1,1,1,1,...,0,1,1,1,1,1,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,0,0,1,1,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,1,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,1,1,1,0,1
7,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,1,0,1,1,0


In [20]:
print('特徴量を{}個選ぶとすると、解の個数は{}通りとなった（入力のyも含む）'.format(num_select, len(E0_Y_lists_df.T.drop_duplicates().T.columns)))

特徴量を3個選ぶとすると、解の個数は3141通りとなった（入力のyも含む）


In [21]:
Y_lists_lastE0_df = pd.DataFrame()
for t in range(len_T):
    if E_lists[t][-1] == 0:
        Y_lists_lastE0_df[t+1]=Y_lists[t]
        
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]

Y_lists_lastE0_df['ori']=ori_y

Y_lists_lastE0_df.index.name = 'y'
Y_lists_lastE0_df.columns.name = 'temperature'
Y_lists_lastE0_df

temperature,333,758,795,1026,1051,1343,1785,2009,2076,2233,2473,2546,2652,3135,3251,3704,4813,ori
y,,,,,,,,,,,,,,,,,,
0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1
1,1,0,0,0,1,0,0,1,1,1,0,1,0,1,1,1,1,0
2,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,0,1
4,0,1,1,0,0,0,0,0,1,0,1,0,1,1,1,1,1,1
5,1,1,1,1,1,0,1,1,0,1,1,1,1,0,0,1,0,0
6,1,0,0,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0
7,0,1,1,1,0,1,0,0,1,0,1,0,1,0,0,1,1,1
8,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,0,1,0
